In [5]:
import random
import pandas as pd
from faker import Faker
from datetime import datetime, timedelta
import os

fake = Faker()

# Batch configuration
batches = {
    1: {"start": "2023-01-01", "end": "2023-05-31", "batch_cap": 50, "tracks": [1, 2, 3, 4]},
    2: {"start": "2023-06-01", "end": "2023-10-31", "batch_cap": 100, "tracks": [1, 2, 3, 4]},
    3: {"start": "2023-11-01", "end": "2024-03-31", "batch_cap": 150, "tracks": [1, 2, 3, 4]},
    4: {"start": "2024-05-01", "end": "2024-09-30", "batch_cap": 200, "tracks": [1, 2, 3, 4]},
    5: {"start": "2024-10-01", "end": "2025-02-28", "batch_cap": 250, "tracks": [1, 2, 3, 4, 5, 6]},
    6: {"start": "2025-03-01", "end": "2025-07-31", "batch_cap": 300, "tracks": [1, 2, 3, 4, 5, 6]},
}

user_ids = list(range(1, 501))
rows = []
payment_rows = []

track_enrollments = {(b, t): set() for b in batches for t in batches[b]["tracks"]}
batch_enrollments = {b: set() for b in batches}
track_max = 200
total_enrollments = 0
max_enrollments = 750

payment_counter = 1
user_batch_payment = {}

random.shuffle(user_ids)

for user_id in user_ids:
    batch_choices = list(batches.keys())
    random.shuffle(batch_choices)
    enrolled_batches = 0

    for batch_id in batch_choices:
        if enrolled_batches >= 3 or total_enrollments >= max_enrollments:
            break

        batch = batches[batch_id]
        if len(batch_enrollments[batch_id]) >= batch["batch_cap"]:
            continue

        available_tracks = [t for t in batch["tracks"] if len(track_enrollments[(batch_id, t)]) < track_max]
        if not available_tracks:
            continue

        random.shuffle(available_tracks)
        selected_tracks = available_tracks[:random.randint(1, min(3, len(available_tracks)))]

        if not selected_tracks:
            continue

        batch_enrollments[batch_id].add(user_id)
        payment_id = f"PAY{payment_counter:05d}"
        user_batch_payment[(user_id, batch_id)] = {
            "payment_id": payment_id,
            "track_count": len(selected_tracks)
        }
        payment_counter += 1
        enrolled_batches += 1
        total_enrollments += 1

        start_date = datetime.strptime(batch["start"], "%Y-%m-%d")
        reg_date = fake.date_between(start_date=start_date - timedelta(days=60),
                                     end_date=start_date - timedelta(days=1))
        status = 'incomplete' if batch_id == 6 else random.choice(['complete', 'incomplete'])

        for track_id in selected_tracks:
            track_enrollments[(batch_id, track_id)].add(user_id)
            rows.append({
                "USER_ID": user_id,
                "TRACK_ID": track_id,
                "PAYMENT_ID": payment_id,
                "BATCH_ID": batch_id,
                "REGISTRATION_DATE": reg_date.strftime("%Y-%m-%d"),
                "STATUS": status
            })

        # Payment table details
        method = random.choice([1, 2])
        if method == 1:
            payment_type = "online"
        else:
            payment_type = random.choice(["visa", "mastercard"])

        payment_rows.append({
            "PAYMENT_ID": payment_id,
            "USER_ID": user_id,
            "AMOUNT": 50 if len(selected_tracks) == 1 else 100,
            "STATUS": "paid",
            "PAYMENT_REFERENCE": f"images/payment_{random.randint(1000, 9999)}.png",
            "NOTES": fake.sentence(),
            "PAYMENT_DATE": reg_date.strftime("%Y-%m-%d"),
            "PAYMENT_METHOD": method,
            "PAYMENT_TYPE": payment_type
        })

# Create DataFrames
df_main = pd.DataFrame(rows)
df_payment = pd.DataFrame(payment_rows)

# Save to Excel
desktop_path = os.path.join(os.path.expanduser("~"), "Desktop")
file_path = os.path.join(desktop_path, "user_enrollments_and_payments.xlsx")

with pd.ExcelWriter(file_path) as writer:
    df_main.to_excel(writer, sheet_name="User_Registrations", index=False)
    df_payment.to_excel(writer, sheet_name="Payments", index=False)

print(f"✅ Excel file saved at: {file_path}")


✅ Excel file saved at: C:\Users\Charbel\Desktop\user_enrollments_and_payments.xlsx
